<a href="https://colab.research.google.com/github/UTKARSH4/Sentiment-Analysis-Using-BERT/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:

!pip install transformers
import transformers
from transformers import PreTrainedModel, PreTrainedTokenizer, PretrainedConfig
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig, BertModel
from transformers import DistilBertTokenizer, DistilBertModel, TFBertModel

from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig, DistilBertModel
from tensorflow.keras import Sequential,Input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
import keras

In [11]:
PRE_TRAINED_MODEL_NAME = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
bert_model = DistilBertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

bert_embeddings = bert_model.get_input_embeddings()

In [51]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
batch_1 = df[:2000]
batch_1[0]=batch_1[0].apply(lambda x:x.split())
for i in range(batch_1.shape[0]):
  batch_1[0][i].insert(0,"[CLS]")
  batch_1[0][i].append("[SEP]")





/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [54]:

tokenizer=Tokenizer()
tokenizer.fit_on_texts(batch_1[0])
data_sequences = tokenizer.texts_to_sequences(batch_1[0])
data_padded = pad_sequences(data_sequences, maxlen=150, padding='post', truncating='post')


In [55]:
attention_mask = np.where(data_padded != 0, 1, 0)

max_len=150
input = tf.convert_to_tensor(data_padded, dtype=tf.int32)
#input_type_id = Input(shape=(max_len,), dtype=tf.int32)
attention_mask_tensor = tf.convert_to_tensor(attention_mask, dtype=tf.int32)
encoder = BertModel.from_pretrained("bert-base-uncased")
last_hidden_states = encoder(input, attention_mask=attention_mask_tensor)

AttributeError: ignored

In [54]:

start_logits = layers.Dense(1, name="start_logit", use_bias=False)(last_hidden_states[0])
start_logits = layers.Flatten()(start_logits)

end_logits = layers.Dense(1, name="end_logit", use_bias=False)(last_hidden_states[0])
end_logits = layers.Flatten()(end_logits)

start_probs = layers.Activation(keras.activations.softmax)(start_logits)
end_probs = layers.Activation(keras.activations.softmax)(end_logits)

model = keras.Model(
    inputs=[input, attention_mask_tensor],
    outputs=[start_probs, end_probs],
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = keras.optimizers.Adam(lr=5e-5)
model.compile(optimizer=optimizer, loss=[loss, loss])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, 150)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   input_14[0][0]                   
                                                                 input_16[0][0]                   
__________________________________________________________________________________________________
start_logit (Dense)             (None, 150, 1)       768         tf_bert_model[0][0]        

'[SEP]'